In [1]:
# Yernar Shambayev, DL-2
# Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
# и задавались следующим образом:
# x - последовательность цифр
# y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10
#
# Задача:
# научить модель предсказывать y(i) по x(i)

import torch
import torch.nn as nn
from random import randint

In [2]:
num_sequences = 10
num_elements = 50

def compute_y(x, shift):
    y = x + shift
    if y >= 10:
        y -= 10
    return y

def generate_sequences():
    seq_in = []
    seq_out = []

    for i in range(num_sequences):
        for j in range(num_elements):
            if j == 0:
                x1 = y1 = randint(1, 9)

                x_ = [[]]
                y_ = []
                x_[0].append(x1)
                y_.append(y1)
            else:
                x = randint(0, 9)
                y = compute_y(x, x1)

                x_[0].append(x)
                y_.append(y)

        seq_in.append(x_)
        seq_out.append(y_)

    return seq_in, seq_out

X_train, Y_train = generate_sequences()

In [3]:
print(X_train[0:2])
print(Y_train[0:2])

[[[1, 3, 5, 8, 5, 6, 0, 5, 1, 6, 9, 8, 7, 9, 2, 2, 1, 0, 7, 0, 5, 1, 3, 5, 7, 6, 8, 4, 4, 9, 1, 6, 2, 0, 6, 1, 3, 3, 3, 0, 9, 6, 7, 8, 8, 1, 6, 4, 3, 1]], [[4, 3, 9, 4, 1, 2, 9, 9, 6, 2, 2, 7, 7, 5, 4, 3, 5, 7, 1, 6, 0, 3, 4, 3, 9, 2, 8, 4, 5, 8, 0, 4, 2, 8, 4, 6, 2, 4, 2, 1, 9, 5, 8, 6, 5, 2, 3, 0, 6, 9]]]
[[1, 4, 6, 9, 6, 7, 1, 6, 2, 7, 0, 9, 8, 0, 3, 3, 2, 1, 8, 1, 6, 2, 4, 6, 8, 7, 9, 5, 5, 0, 2, 7, 3, 1, 7, 2, 4, 4, 4, 1, 0, 7, 8, 9, 9, 2, 7, 5, 4, 2], [4, 7, 3, 8, 5, 6, 3, 3, 0, 6, 6, 1, 1, 9, 8, 7, 9, 1, 5, 0, 4, 7, 8, 7, 3, 6, 2, 8, 9, 2, 4, 8, 6, 2, 8, 0, 6, 8, 6, 5, 3, 9, 2, 0, 9, 6, 7, 4, 0, 3]]


In [4]:
num_layers = 1  
embedding_size = 10  
num_classes = 10
input_size = 10
batch_size = 1  
sequence_length = num_elements  
hidden_size = 10  

from torch.autograd import Variable

class RNN_Model(nn.Module):
    def __init__(self):
        super(RNN_Model, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)

        out, _ = self.rnn(emb, h_0)
        return self.fc(out.view(-1, num_classes))

In [5]:
model = RNN_Model()
print(model)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

RNN_Model(
  (embedding): Embedding(10, 10)
  (rnn): RNN(10, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=10, bias=True)
)


In [6]:
str_sequence = [str(i) for i in range(10)]
print(str_sequence)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [7]:
# Обучение и проверка
epochs = 50
for i in range(num_sequences):
    print(f'{i+1} sequence')
    inputs = Variable(torch.LongTensor(X_train[i]))
    labels = Variable(torch.LongTensor(Y_train[i]))

    for epoch in range(epochs):
        outputs = model(inputs)
        optimizer.zero_grad()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 1:
            print(f"Эпоха: {epoch + 1}, потери: {loss.item():1.3f}")

        if epoch == epochs - 1: # проверка в конце эпох
            _, idx = outputs.max(1)
            idx = idx.data.numpy()
            result_str = [str_sequence[c] for c in idx.squeeze()]
            orig_str  = [str_sequence[c] for c in inputs[0]]
            label_str  = [str_sequence[c] for c in Y_train[i]]

            print("Входная последовательность:  ", ''.join(orig_str))
            print("Выходная последовательность: ", ''.join(label_str))
            print("Предсказание:                ", ''.join(result_str))
            print("")

1 sequence
Эпоха: 1, потери: 2.326
Эпоха: 11, потери: 0.103
Эпоха: 21, потери: 0.056
Эпоха: 31, потери: 0.025
Эпоха: 41, потери: 0.009
Входная последовательность:   13585605169879221070513576844916206133309678816431
Выходная последовательность:  14696716270980332181624687955027317244410789927542
Предсказание:                 14696716270980332181624687955027317244410789927542

2 sequence
Эпоха: 1, потери: 12.090
Эпоха: 11, потери: 1.081
Эпоха: 21, потери: 0.166
Эпоха: 31, потери: 0.078
Эпоха: 41, потери: 0.051
Входная последовательность:   43941299622775435716034392845804284624219586523069
Выходная последовательность:  47385633066119879150478736289248628068653920967403
Предсказание:                 47385633066119879150478736289248628068653920967403

3 sequence
Эпоха: 1, потери: 13.354
Эпоха: 11, потери: 2.183
Эпоха: 21, потери: 0.394
Эпоха: 31, потери: 0.213
Эпоха: 41, потери: 0.092
Входная последовательность:   64270756114629051536252258633661006749686632761497
Выходная последовательно